In [2]:
# --- Librerías ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, Layout

# --- Cargar archivo CSV ---
RUTA = '/workspaces/ml-bootcamp-labs/data/raw/Shift_Pozos_LDiscEoc-cvs.csv'
df = pd.read_csv(RUTA, sep=';')

# Normalizar nombres de columnas
df.columns = df.columns.str.strip().str.lower()

# Columnas clave
col_delta = 'delta'
col_abs_delta = 'absolute delta'
col_x = 'x'
col_y = 'y'
col_pickz = 'pick z'

# --- Función de estadísticos personalizados ---
def stats(x):
    return pd.Series({
        'count': x.count(),
        'mean': x.mean(),
        'std': x.std(),
        'min': x.min(),
        'p10': np.percentile(x, 10),
        'p25': np.percentile(x, 25),
        'p50': np.percentile(x, 50),
        'p75': np.percentile(x, 75),
        'p90': np.percentile(x, 90),
        'max': x.max()
    })

# --- Función principal de análisis ---
def analisis(pozos_seleccionados):
    if not pozos_seleccionados:
        print("Selecciona al menos un pozo.")
        return
    
    # Filtrar pozos seleccionados
    df_sel = df[df['uwi'].isin(pozos_seleccionados)]
    
    # Estadísticos agregados por pozo
    stats_group = df_sel.groupby('uwi')[col_delta].apply(stats).reset_index()
    display(stats_group)
    
    # --- Histogramas por pozo ---
    plt.figure(figsize=(10,6))
    sns.histplot(data=df_sel, x=col_delta, hue='uwi', multiple='stack', bins=20)
    plt.title("Histograma de Delta por Pozo")
    plt.xlabel("Delta")
    plt.ylabel("Frecuencia")
    plt.show()
    
    # --- Boxplot por pozo ---
    plt.figure(figsize=(10,6))
    sns.boxplot(x='uwi', y=col_delta, data=df_sel)
    plt.title("Boxplot de Delta por Pozo")
    plt.xlabel("Pozo (UWI)")
    plt.ylabel("Delta")
    plt.show()
    
    # --- Violin plot por pozo ---
    plt.figure(figsize=(10,6))
    sns.violinplot(x='uwi', y=col_delta, data=df_sel)
    plt.title("Violin plot de Delta por Pozo")
    plt.xlabel("Pozo (UWI)")
    plt.ylabel("Delta")
    plt.show()
    
    # --- Mapa espacial ---
    plt.figure(figsize=(8,6))
    sc = plt.scatter(df_sel[col_x], df_sel[col_y], c=df_sel[col_delta], cmap='RdBu', s=60, edgecolor='k')
    plt.colorbar(sc, label="Delta")
    plt.title("Distribución espacial de Delta")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()
    
    # --- Delta vs Pick Z ---
    plt.figure(figsize=(8,6))
    sns.scatterplot(x=df_sel[col_pickz], y=df_sel[col_delta], hue=df_sel['uwi'])
    plt.title("Delta vs Profundidad (Pick Z)")
    plt.xlabel("Pick Z")
    plt.ylabel("Delta")
    plt.axhline(0, color='red', linestyle='--')
    plt.show()
    
    # --- Exportar estadísticas ---
    stats_group.to_excel("estadisticas_shifts_por_pozos.xlsx", index=False)
    print("Archivo 'estadisticas_shifts_por_pozos.xlsx' exportado con éxito.")

# --- Widget de selección múltiple de pozos ---
pozo_selector = widgets.SelectMultiple(
    options=df['uwi'].dropna().unique(),
    description='Pozos:',
    layout=Layout(width='50%', height='200px')
)

# --- Interacción ---
interact(analisis, pozos_seleccionados=pozo_selector)


interactive(children=(SelectMultiple(description='Pozos:', layout=Layout(height='200px', width='50%'), options…

<function __main__.analisis(pozos_seleccionados)>